# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.235636015073                   -0.50    8.0
  2   -7.250454673718       -1.83       -1.40    1.0
  3   -7.251319359605       -3.06       -2.13    4.0
  4   -7.251310099473   +   -5.03       -2.39    4.0
  5   -7.251325930644       -4.80       -2.60    3.0
  6   -7.251337643055       -4.93       -3.11    2.0
  7   -7.251338767763       -5.95       -4.04    1.0
  8   -7.251338787245       -7.71       -4.02    4.0
  9   -7.251338797421       -7.99       -4.44    3.0
 10   -7.251338798018       -9.22       -4.62    1.0
 11   -7.251338798697       -9.17       -5.80    3.0
 12   -7.251338798704      -11.11       -6.03    4.0
 13   -7.251338798704      -12.77       -6.43    2.0
 14   -7.251338798705      -13.34       -6.79    2.0
 15   -7.251338798705      -13.94       -7.24    2.0
 16   -7.251338798705   +  -14.75       -7.64    3.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.057839899012218546
[ Info: Arnoldi iteration step 2: normres = 0.6173625823732082
[ Info: Arnoldi iteration step 3: normres = 0.8199902051170637
[ Info: Arnoldi iteration step 4: normres = 0.2527430576220875
[ Info: Arnoldi iteration step 5: normres = 0.4677976123211599
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.20e-02, 6.63e-02, 4.17e-01, 1.99e-01, 9.70e-03)
[ Info: Arnoldi iteration step 6: normres = 0.3357890757323083
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (6.53e-03, 1.96e-01, 2.22e-01, 1.16e-01, 6.29e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07716241934080051
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.52e-04, 1.68e-02, 9.92e-03, 4.48e-02, 5.26e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10008242857247779
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.07e-